In [1]:
#coding:utf-8
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.cross_validation import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

#read data
train = pd.read_csv("C:\\Users\\hasee\\workspace\\Kaggle\\Titanic\\train.csv")
test = pd.read_csv("C:\\Users\\hasee\\workspace\\Kaggle\\Titanic\\test.csv")

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def clean_data(titanic):#填充空数据 和 把string数据转成integer表示
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    # child
    titanic["child"] = titanic["Age"].apply(lambda x: 1 if x < 15 else 0)

    # sex
    titanic["sex"] = titanic["Sex"].apply(lambda x: 1 if x == "male" else 0)

    titanic["Embarked"] = titanic["Embarked"].fillna("S")
    # embark
    def getEmbark(Embarked):
        if Embarked == "S":
            return 1
        elif Embarked == "C":
            return 2
        else:
            return 3
    titanic["embark"] = titanic["Embarked"].apply(getEmbark)

    # familysize
    titanic["fimalysize"] = titanic["SibSp"] + titanic["Parch"] + 1

    # cabin
    def getCabin(cabin):
        if cabin == "N":
            return 0
        else:
            return 1
    titanic["cabin"] = titanic["Cabin"].apply(getCabin)

    # name
    def getName(name):
        if "Mr" in str(name):
            return 1
        elif "Mrs" in str(name):
            return 2
        else:
            return 0
    titanic["name"] = titanic["Name"].apply(getName)

    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic
# 对数据进行清洗
train_data = clean_data(train)
test_data = clean_data(test)

In [3]:
features = ["Pclass", "sex", "child", "fimalysize", "Fare", "embark", "cabin"]

In [4]:
forest_clf=RandomForestClassifier(max_features='sqrt')

In [6]:
param_grid = {
    'n_estimators': range(100, 400, 50),
    'max_depth': range(2, 7, 1),
    'criterion':['gini','entropy']
}
grid_clf = GridSearchCV(estimator = forest_clf, param_grid = param_grid, scoring='accuracy', cv=5)
grid_clf.fit(train_data[features], train_data["Survived"])
grid_clf.grid_scores_, grid_clf.best_params_, grid_clf.best_score_

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.79574, std: 0.02213, params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 100},
  mean: 0.79461, std: 0.02900, params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 150},
  mean: 0.79012, std: 0.02387, params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 200},
  mean: 0.79125, std: 0.03105, params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 250},
  mean: 0.79349, std: 0.02269, params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 300},
  mean: 0.79125, std: 0.02816, params: {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 350},
  mean: 0.81033, std: 0.01198, params: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 100},
  mean: 0.81481, std: 0.02390, params: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 150},
  mean: 0.80920, std: 0.03805, params: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 200},
  mean: 0.81594, std: 0.02494, params: {'criterion': 'gini', 'max_depth': 3, 'n_estimators': 250},
  mean: 0.

In [7]:
y_pred=grid_clf.best_estimator_.predict(test_data[features])
submission=pd.DataFrame(test_data["PassengerId"])
submission.insert(1,'Survived',y_pred)
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [8]:
submission.to_csv("C:\\Users\\hasee\\workspace\\Kaggle\\Titanic\\submission3.csv")